In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define dataset file paths
datasets = {
    "gas_price": "gas_price.csv",
    "rig_count": "rig_count.csv",
    "tam": "tam.csv",
    "capex": "capex.csv",
    "oil_price": "oil_price.csv",
    "education": "education.csv",
    "political_stability": "political_stability.csv",
    "infrastructure_index": "infrastructure_index.csv",
    "health_index": "health_index.csv",
    "gni": "gni.xlsx",
    "gdp": "gdp.xlsx",
    "population": "population.xlsx",
    "country_mapping": "country_table(in).csv" 
}

def load_data(dataset_name):
    """Loads and processes the dataset."""
    path = datasets.get(dataset_name)
    if not path:
        print(f"Dataset '{dataset_name}' not found in provided paths.")
        return None

    try:
        if path.lower().endswith('.csv'):
            df = pd.read_csv(path)
        elif path.lower().endswith(('.xlsx', '.xls')):
            df = pd.read_excel(path)
        else:
            print(f"Unsupported file format for dataset '{dataset_name}'.")
            return None

        df["country_name"] = df["country_name"].str.strip().str.upper()
        df["year"] = df["year"].astype(int)
        return df
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        return None

        
# ---------- Model Building ----------
def build_nn_model(input_dim):
    """Builds and returns a neural network model."""
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# ---------- Training Country-Specific Models (rig_count, tam, capex) ----------
def train_model_for_country(dataset_name, country):
    """Trains a neural network model for a specific country and dataset."""
    df = load_data(dataset_name)
    if df is None:
        print(f"Skipping {country}: Unable to load data for {dataset_name}.")
        return None, None, None, None

    country_df = df[df["country_name"] == country]
    if country_df.empty:
        print(f"Skipping {country}: No data available for {dataset_name}.")
        return None, None, None, None

    # Assume the target feature is the one that is not 'country_name' or 'year'
    target_feature = [col for col in country_df.columns if col in selected_options or col not in ["country_name", "year"]][0]
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Sort and interpolate missing data
    country_df_sorted = country_df.sort_values("year")
    country_df_sorted.interpolate(method="linear", inplace=True)

    X = country_df_sorted.drop(columns=["country_name", "year"])
    y = country_df_sorted[[target_feature]]

    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y)

    if len(X_scaled) < 5:
        print(f"Skipping {country}: Not enough data for training in {dataset_name}.")
        return None, None, None, None

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=X_train.shape[1])

    # Callbacks for training
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[lr_scheduler, early_stopping], verbose=2)

    return model, scaler, country_df_sorted, target_feature

# ---------- Training Year-Based Models (gas_price, oil_price) ----------
def train_nn_model(df, target_feature):
    """Trains a neural network model using 'year' as the feature."""
    grouped_df = df.groupby("year").mean(numeric_only=True).reset_index()
    X = grouped_df[["year"]].values
    y = grouped_df[[target_feature]].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    model = build_nn_model(input_dim=1)

    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32,
              callbacks=[lr_scheduler, early_stopping], verbose=2)

    return model, scaler_X, scaler_y

# ---------- Prediction Functions ----------
def predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country):
    """
    Predicts future values using a country-specific model.
    The returned DataFrame will always use 'year' and 'country_name' as keys.
    If the target is 'rig_count', the prediction is rounded to a whole number.
    """
    if model is None:
        return pd.DataFrame()

    future_predictions = []
    latest_data = df[df["year"] == df["year"].max()]
    latest_scaled = scaler.transform(latest_data.drop(columns=["country_name", "year"]))

    for i in range(num_years):
        predicted_scaled = model.predict(latest_scaled, verbose=0)
        predicted_value = scaler.inverse_transform(predicted_scaled)[0][0]
        trend_factor = np.random.uniform(0.9, 1.1)
        predicted_value *= trend_factor

        if target_feature == "rig_count":
            predicted_value = int(round(predicted_value))

        prediction_year = start_year + i
        future_predictions.append({
            "year": prediction_year,
            "country_name": country,
            target_feature: predicted_value
        })
        print(f"Predicted {target_feature} for {country} in {prediction_year}: {predicted_value}")

    return pd.DataFrame(future_predictions)

def predict_prices(model, scaler_X, scaler_y, start, end, countries, feature_name):
    """
    Predicts prices using a year-based model.
    Returns a DataFrame with 'year' and 'country_name' columns.
    """
    predictions = []
    years = np.arange(start, end + 1).reshape(-1, 1)
    years_scaled = scaler_X.transform(years)
    preds_scaled = model.predict(years_scaled, verbose=0)
    preds = scaler_y.inverse_transform(preds_scaled)

    for country in countries:
        for year, price in zip(years.flatten(), preds.flatten()):
            predictions.append({
                "year": year,
                "country_name": country,
                feature_name: price
            })
    return pd.DataFrame(predictions)
    

# --------------------------------- Main Execution Flow -------------------------------------

# Get dataset names (comma separated)
valid_datasets = {"gas_price", "oil_price", "rig_count", "tam", "capex"}
dataset_names_input = input("\nEnter what you want to predict (gas_price, oil_price, rig_count, tam, capex): ")
dataset_names = []

for name in dataset_names_input.split(','):
    clean_name = name.strip()
    if clean_name in valid_datasets:
        dataset_names.append(clean_name)

for i, dataset_name in enumerate(datasets.keys(), start=1):
    print(f"{i}. {dataset_name}")
# Menu Display (Select Datasets for Prediction)
print("\nAvailable Datasets for Prediction, Here's a catch - If you select all relevant data then predictions is on basis of global indicators else the neural network identify correlation itself adding historical considerations to predict\n")
print("1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.")
print("2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.")
print("3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.")
print("4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.")
print("5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.")

selected_options = input("\nEnter the numbers corresponding to the datasets you want to predict: ").strip()

# Load country mapping to get available countries
gas_price_data = load_data("gas_price")
if gas_price_data is not None and "country_name" in gas_price_data.columns:
    available_countries = sorted(gas_price_data["country_name"].unique())
    print("\nAvailable countries:", ", ".join(available_countries))
else:
    print("Gas price data could not be loaded or does not contain country information.")
    available_countries = []


# Get countries from user input
selected_countries_input = input("Enter countries (comma-separated): ")
selected_countries = [c.strip().upper() for c in selected_countries_input.split(',')
                      if c.strip().upper() in available_countries]

# Get prediction year range from user
start_year = int(input("Enter the start year for predictions: "))
end_year = int(input("Enter the end year for predictions: "))

# Train global models for gas and oil prices
oil_price_data = load_data("oil_price")
gas_model, gas_scaler_X, gas_scaler_y = train_nn_model(gas_price_data, "gas_price")
oil_model, oil_scaler_X, oil_scaler_y = train_nn_model(oil_price_data, "oil_price")

# Create a skeleton DataFrame containing all combinations of selected countries and years
years = list(range(start_year, end_year + 1))
skeleton = pd.DataFrame([(year, country) for country in selected_countries for year in years],
                        columns=["year", "country_name"])
final_predictions = skeleton.copy()

# Process each dataset's predictions and merge them into final_predictions
for dataset in dataset_names:
    if dataset in ["gas_price", "oil_price"]:
        # Global models predict for all countries at once.
        if dataset == "gas_price" and gas_model:
            pred_df = predict_prices(gas_model, gas_scaler_X, gas_scaler_y, start_year, end_year,
                                      selected_countries, "gas_price")
        elif dataset == "oil_price" and oil_model:
            pred_df = predict_prices(oil_model, oil_scaler_X, oil_scaler_y, start_year, end_year,
                                      selected_countries, "oil_price")
        else:
            pred_df = pd.DataFrame()
        final_predictions = pd.merge(final_predictions, pred_df, on=["year", "country_name"], how="left")
    elif dataset in ["rig_count", "tam", "capex"]:
        # Country-specific models: train and predict per country.
        all_preds = []
        for country in selected_countries:
            model, scaler, df, target_feature = train_model_for_country(dataset, country)
            if model:
                num_years = end_year - start_year + 1
                pred_df = predict_future_by_country(model, scaler, df, target_feature, start_year, num_years, country)
                all_preds.append(pred_df)
        if all_preds:
            pred_all = pd.concat(all_preds, ignore_index=True)
            final_predictions = pd.merge(final_predictions, pred_all, on=["year", "country_name"], how="left")

def load_country_mapping(file_path):
    """Loads and preprocesses country mappings from the provided CSV file."""
    try:
        country_mapping = pd.read_csv(file_path)
        country_mapping["country_name"] = country_mapping["country_name"].str.strip().str.upper()
        return country_mapping
    except Exception as e:
        print(f"Error loading country mapping: {e}")
        return None

country_mapping = load_country_mapping(datasets["country_mapping"])

if country_mapping is not None and "country_name" in country_mapping.columns and "country_id" in country_mapping.columns:
    final_predictions = pd.merge(final_predictions, country_mapping[["country_name", "country_id"]], on="country_name", how="left")

    # Reorder columns
    columns_order = ["country_id", "country_name", "year"] + [col for col in final_predictions.columns if col not in ["year", "country_name", "country_id"]]
    final_predictions = final_predictions[columns_order]
else:
    print("Country mapping data is invalid or missing required columns.")

# Save predictions
output_file = input("\nEnter the file name to save predictions (e.g., predictions_output.csv): ").strip()
output_file += ".csv" if not output_file.lower().endswith(".csv") else ""

try:
    final_predictions.to_csv(output_file, index=False)
    print(f"\nPredictions have been successfully saved to '{output_file}'.")
except Exception as e:
    print(f"Error saving predictions: {e}")

1. gas_price
2. rig_count
3. tam
4. capex
5. oil_price
6. education
7. political_stability
8. infrastructure_index
9. health_index
10. gni
11. gdp
12. population
13. country_mapping

Available Datasets for Prediction, Here's a catch - If you select all relevant data then predictions is on basis of global indicators else the neural network identify correlation itself adding historical considerations to predict

1. Gas Price Prediction: Uses Oil Price, Rig Count, Infrastructure Index, Capex.
2. Rig Count Prediction: Uses Rig Count, Infrastructure Index, Capex, Political Stability.
3. Oil Price Prediction: Uses Rig Count, GDP Growth, Infrastructure Index, Capex, Political Stability.
4. Tam Prediction: Uses Capex, GDP, Population, Health Index, Education, Infrastructure Index.
5. Capex Prediction: Uses Oil Price, Gas Price, Rig Count, GDP Growth, Political Stability, Infrastructure Index.

Available countries: AFGHANISTAN, ALBANIA, ALGERIA, ANDORRA, ANGOLA, ANTIGUA & BARBUDA, ARGENTINA, AR

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 - 6s - 6s/step - loss: 3.2902 - mae: 1.5119 - val_loss: 0.2890 - val_mae: 0.4418 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 287ms/step - loss: 1.4806 - mae: 1.0377 - val_loss: 0.2813 - val_mae: 0.4324 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 329ms/step - loss: 1.8041 - mae: 1.0239 - val_loss: 0.2741 - val_mae: 0.4239 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 264ms/step - loss: 3.3200 - mae: 1.6107 - val_loss: 0.2667 - val_mae: 0.4143 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 274ms/step - loss: 1.6403 - mae: 1.0170 - val_loss: 0.2592 - val_mae: 0.4044 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 292ms/step - loss: 1.5099 - mae: 0.9267 - val_loss: 0.2535 - val_mae: 0.3964 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 286ms/step - loss: 4.4967 - mae: 1.3430 - val_loss: 0.2476 - val_mae: 0.3876 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 279ms/step - loss: 1.3824 - mae: 0.9547 - val_loss: 0.2422 - val_mae: 0.3788 - learning_rate: 0.0010
Epoch 9/50
1/1 - 0s

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 - 6s - 6s/step - loss: 2.0282 - mae: 1.0119 - val_loss: 0.2165 - val_mae: 0.3265 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 312ms/step - loss: 1.9248 - mae: 1.0299 - val_loss: 0.2182 - val_mae: 0.3304 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 328ms/step - loss: 1.8508 - mae: 1.1677 - val_loss: 0.2170 - val_mae: 0.3307 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 275ms/step - loss: 1.5842 - mae: 0.9324 - val_loss: 0.2144 - val_mae: 0.3291 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 292ms/step - loss: 1.1119 - mae: 0.8241 - val_loss: 0.2112 - val_mae: 0.3268 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 325ms/step - loss: 2.6851 - mae: 1.3825 - val_loss: 0.2077 - val_mae: 0.3236 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 289ms/step - loss: 0.9585 - mae: 0.8730 - val_loss: 0.2048 - val_mae: 0.3211 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 282ms/step - loss: 1.1581 - mae: 0.9406 - val_loss: 0.2012 - val_mae: 0.3177 - learning_rate: 0.0010
Epoch 9/50
1/1 - 0s

...

Epoch 1/50
1/1 - 6s - 6s/step - loss: 1.8623 - mae: 1.0336 - val_loss: 0.1213 - val_mae: 0.2662 - learning_rate: 0.0010
Epoch 2/50
1/1 - 1s - 751ms/step - loss: 1.7274 - mae: 1.0817 - val_loss: 0.1204 - val_mae: 0.2670 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 332ms/step - loss: 2.8287 - mae: 1.1587 - val_loss: 0.1224 - val_mae: 0.2721 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 312ms/step - loss: 1.7972 - mae: 0.9692 - val_loss: 0.1235 - val_mae: 0.2772 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 293ms/step - loss: 2.7365 - mae: 1.3363 - val_loss: 0.1246 - val_mae: 0.2827 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 319ms/step - loss: 1.2824 - mae: 0.8956 - val_loss: 0.1254 - val_mae: 0.2873 - learning_rate: 0.0010
Epoch 7/50

Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1/1 - 0s - 280ms/step - loss: 1.7071 - mae: 0.9773 - val_loss: 0.1260 - val_mae: 0.2916 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 303ms/step - loss: 2.2053 - mae: 1.2003 - v

C:\Users\HP\AppData\Local\Temp\ipykernel_10148\3858047369.py:85: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  country_df_sorted.interpolate(method="linear", inplace=True)
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 - 6s - 6s/step - loss: 2.3017 - mae: 1.3412 - val_loss: 0.1495 - val_mae: 0.3005 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 295ms/step - loss: 2.1443 - mae: 1.2814 - val_loss: 0.1546 - val_mae: 0.3057 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 333ms/step - loss: 1.2621 - mae: 0.9809 - val_loss: 0.1548 - val_mae: 0.3060 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 330ms/step - loss: 3.3923 - mae: 1.4109 - val_loss: 0.1514 - val_mae: 0.3064 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 296ms/step - loss: 1.3264 - mae: 0.9423 - val_loss: 0.1473 - val_mae: 0.3055 - learning_rate: 0.0010
Epoch 6/50
1/1 - 0s - 300ms/step - loss: 2.0547 - mae: 1.3362 - val_loss: 0.1438 - val_mae: 0.3045 - learning_rate: 0.0010
Epoch 7/50
1/1 - 0s - 309ms/step - loss: 3.4196 - mae: 1.6466 - val_loss: 0.1407 - val_mae: 0.3037 - learning_rate: 0.0010
Epoch 8/50
1/1 - 0s - 316ms/step - loss: 3.2816 - mae: 1.5989 - val_loss: 0.1372 - val_mae: 0.3027 - learning_rate: 0.0010
Epoch 9/50
1/1 - 0s

C:\Users\HP\AppData\Local\Temp\ipykernel_10148\3858047369.py:85: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  country_df_sorted.interpolate(method="linear", inplace=True)
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 - 6s - 6s/step - loss: 2.1514 - mae: 1.2176 - val_loss: 0.0774 - val_mae: 0.2138 - learning_rate: 0.0010
Epoch 2/50
1/1 - 0s - 311ms/step - loss: 1.7995 - mae: 1.1061 - val_loss: 0.0809 - val_mae: 0.2190 - learning_rate: 0.0010
Epoch 3/50
1/1 - 0s - 330ms/step - loss: 2.4099 - mae: 1.2586 - val_loss: 0.0847 - val_mae: 0.2243 - learning_rate: 0.0010
Epoch 4/50
1/1 - 0s - 267ms/step - loss: 2.2626 - mae: 1.3057 - val_loss: 0.0884 - val_mae: 0.2291 - learning_rate: 0.0010
Epoch 5/50
1/1 - 0s - 303ms/step - loss: 2.5945 - mae: 1.3492 - val_loss: 0.0922 - val_mae: 0.2352 - learning_rate: 0.0010
Epoch 6/50

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1/1 - 0s - 287ms/step - loss: 2.0024 - mae: 1.1994 - val_loss: 0.0948 - val_mae: 0.2404 - learning_rate: 0.0010
Epoch 7/50
1/1 - 1s - 609ms/step - loss: 1.4674 - mae: 0.9782 - val_loss: 0.0982 - val_mae: 0.2442 - learning_rate: 5.0000e-04
Epoch 8/50
1/1 - 0s - 351ms/step - loss: 1.3955 - mae: 1.0289